In [7]:
import numpy as np
import h5py
import neo
import quantities as pq
def LvR(t):
    """
    Returns the LvR for a given set of spike timestamps, t.
    Adapted from: 'Relating Neuronal Firing Patterns to Functional
    Differentiation of Cerebral Cortex.' Shinomoto et al. (2009)
    
    Parameters:
    t : numpy array
        1D array of spike timestamps (seconds)
    
    Returns:
    LvR_out : float
        LvR value for the given segment of a spike train.
    """
    
    # Parameters
    R = 0.0015  # Refractoriness (s)
    # Ensure t is a 1D array
    t = np.ravel(t)
    # Calculate interspike intervals
    I = np.diff(t)
    n = len(I)  # Number of interspike intervals
    if n < 10:  # Check if there are enough intervals to proceed
        return np.nan
    # Calculate the summand using vectorized operations
    I1 = I[:-1]
    I2 = I[1:]
    term1 = (1 - 4 * I1 * I2 / ((I1 + I2) ** 2))
    term2 = (1 + 4 * R / (I1 + I2))
    summand = np.sum(term1 * term2)
    # Normalization constant
    norm = 3 / (n - 1)
    # Output LvR value
    LvR_out = norm * summand 
    
    return LvR_out

filename = './templates/spiketrains.h5'
duration = 120  # stesso t_stop usato prima



spike_trains = []
with h5py.File(filename, 'r') as f:
    for key in f.keys():
        times = f[key][()]  # array di spike times in secondi
        st = neo.SpikeTrain(times * pq.s, t_start=0 * pq.s, t_stop=duration * pq.s)
        spike_trains.append(st)

fs = 32000
Spikes = []
for sp in spike_trains:
    sp_values = sp.rescale('s').magnitude  # solo valori float in secondi
    spikes_samples = (sp_values * fs).astype(int).tolist()
    Spikes.append(spikes_samples)



lvr_values = []

for i, spikes in enumerate(Spikes):
    lvr = LvR(spikes)
    lvr_values.append(lvr)
    print(f"Neurone {i+1}: LvR = {lvr}")

# Se vuoi i risultati come array numpy:
lvr_values = np.array(lvr_values)


Neurone 1: LvR = 1.1740478365306468
Neurone 2: LvR = 1.2066829460753508
Neurone 3: LvR = 1.175175759201981
Neurone 4: LvR = 1.2069343113354847
Neurone 5: LvR = 1.2146687268469145
Neurone 6: LvR = 1.1883444413321778
Neurone 7: LvR = 1.2244237420121125
Neurone 8: LvR = 1.2342533347568712
Neurone 9: LvR = 1.2048134753811635
Neurone 10: LvR = 1.2542808473105842
